# Build a list of Calspec Star for AuxTel

- author : Sylvie Dagoret-Campagne
- creation date : 25/09/2019
- Update : April 20th 2020 for package PhotomCorrectAuxTel


From Pysynphot, il get all CALSPEC filenames. It extract **HD type stars**, get their Ra Dec and filename and make a summary file of all the CALSPEC available

In [1]:
import os
import re
import numpy as np
from math import ceil,floor

In [2]:
import pysynphot as S

In [3]:
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8') # vary depending on your lang/locale

'en_US.UTF-8'

In [4]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
import time
from datetime import datetime,date
import dateutil.parser
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord, EarthLocation, AltAz
from astropy.coordinates import Angle
from astropy import units as u
import astropy.coordinates as coord

In [6]:
from astropy.io import fits
from astropy.table import Table
from astropy.time import Time
import astropy
from astropy.visualization.mpl_normalize import (ImageNormalize,MinMaxInterval,PercentileInterval,SqrtStretch,LogStretch)
from astropy.visualization.wcsaxes import SphericalCircle

In [7]:
from astroquery.simbad import Simbad

In [8]:
pysynphot_root_path=os.environ['PYSYN_CDBS']

In [9]:
pysynphot_root_path

'/Users/dagoret/MacOSX/External/PySynPhotData/grp/hst/cdbs'

In [10]:
path_sed_calspec=os.path.join(pysynphot_root_path,'calspec')

In [11]:
def GetListOfFiles(thedir,tag):
    """
    - thedir : directory where are the files
    - tage  = the re expression to file the filename with regular expression
    """
    all_files=os.listdir(thedir)
    sorted_files=sorted(all_files)
    selected_files=[]
    for sfile in sorted_files:
        if re.search(tag,sfile):
            selected_files.append(sfile)
    return selected_files

In [12]:
def FilterListOfFiles(listOfFiles):
    """
    Filter list of files   
    """
    
    all_selected_files=[]
    
    current_root_fn=None  # root of filename ex hd000000
    current_fn=None    # filename of calspec ex hd000000_stis.fits
    
    for fn in listOfFiles:
        
        root_fn=fn.split("_")[0]
        
        if current_root_fn==None:
            current_root_fn=root_fn
            current_fn=fn
            continue
            
        if root_fn != current_root_fn:
            all_selected_files.append(current_fn)
            
        current_fn=fn
        current_root_fn=root_fn

    return all_selected_files

In [13]:
# the tag to find the good filename
RETAGFNNAME="^hd.*_stis_[0-9][0-9][0-9].fits$"
#RETAGDRNAME="(^T1M_.*_HD116405_Filtre_None)_bin1x1.*_red.fit$"

In [14]:
all_files=os.listdir(path_sed_calspec)

In [15]:
all_files=sorted(all_files)

In [16]:
print(all_files)

['10lac_mod_001.fits', '10lac_stis_001.fits', '10lac_stis_002.fits', '10lac_stis_003.fits', '10lac_stis_004.fits', '1732526_mod_001.fits', '1732526_nic_002.fits', '1732526_stisnic_001.fits', '1732526_stisnic_002.fits', '1732526_stisnic_003.fits', '1732526_stisnic_004.fits', '1740346_nic_001.fits', '1740346_nic_002.fits', '1740346_stisnic_001.fits', '1740346_stisnic_002.fits', '1740346_stisnic_003.fits', '1743045_mod_001.fits', '1743045_nic_001.fits', '1743045_nic_002.fits', '1743045_stisnic_001.fits', '1743045_stisnic_002.fits', '1743045_stisnic_003.fits', '1743045_stisnic_004.fits', '1757132_mod_001.fits', '1757132_stis_001.fits', '1757132_stis_002.fits', '1757132_stis_003.fits', '1757132_stis_004.fits', '1802271_mod_001.fits', '1802271_nic_002.fits', '1802271_stisnic_001.fits', '1802271_stisnic_002.fits', '1802271_stisnic_003.fits', '1802271_stisnic_004.fits', '1805292_mod_001.fits', '1805292_nic_001.fits', '1805292_nic_002.fits', '1805292_stisnic_001.fits', '1805292_stisnic_002.fits

In [17]:
all_hdcalspec=GetListOfFiles(path_sed_calspec,RETAGFNNAME)

In [18]:
print(all_hdcalspec)

['hd009051_stis_001.fits', 'hd009051_stis_002.fits', 'hd009051_stis_003.fits', 'hd031128_stis_001.fits', 'hd031128_stis_002.fits', 'hd031128_stis_003.fits', 'hd074000_stis_001.fits', 'hd074000_stis_002.fits', 'hd074000_stis_003.fits', 'hd106252_stis_001.fits', 'hd106252_stis_002.fits', 'hd106252_stis_003.fits', 'hd106252_stis_004.fits', 'hd111980_stis_001.fits', 'hd111980_stis_002.fits', 'hd111980_stis_003.fits', 'hd116405_stis_001.fits', 'hd116405_stis_002.fits', 'hd116405_stis_003.fits', 'hd116405_stis_004.fits', 'hd14943_stis_001.fits', 'hd14943_stis_002.fits', 'hd14943_stis_003.fits', 'hd14943_stis_004.fits', 'hd158485_stis_001.fits', 'hd158485_stis_002.fits', 'hd158485_stis_003.fits', 'hd158485_stis_004.fits', 'hd159222_stis_001.fits', 'hd159222_stis_002.fits', 'hd159222_stis_003.fits', 'hd159222_stis_004.fits', 'hd160617_stis_001.fits', 'hd160617_stis_002.fits', 'hd160617_stis_003.fits', 'hd163466_stis_001.fits', 'hd163466_stis_002.fits', 'hd163466_stis_003.fits', 'hd163466_stis_

In [19]:
all_selected_calspec=FilterListOfFiles(all_hdcalspec)

In [20]:
print(all_selected_calspec)

['hd009051_stis_003.fits', 'hd031128_stis_003.fits', 'hd074000_stis_003.fits', 'hd106252_stis_004.fits', 'hd111980_stis_003.fits', 'hd116405_stis_004.fits', 'hd14943_stis_004.fits', 'hd158485_stis_004.fits', 'hd159222_stis_004.fits', 'hd160617_stis_003.fits', 'hd163466_stis_004.fits', 'hd180609_stis_004.fits', 'hd185975_stis_003.fits', 'hd200654_stis_003.fits', 'hd205905_stis_004.fits', 'hd37725_stis_004.fits', 'hd37962_stis_004.fits', 'hd38949_stis_004.fits', 'hd60753_stis_003.fits']


In [21]:
Simbad.list_votable_fields()

--NOTES--

1. The parameter filtername must correspond to an existing filter. Filters include: B,V,R,I,J,K.  They are checked by SIMBAD but not astroquery.simbad

2. Fields beginning with rvz display the data as it is in the database. Fields beginning with rv force the display as a radial velocity. Fields beginning with z force the display as a redshift

3. For each measurement catalog, the VOTable contains all fields of the first measurement. When applicable, the first measurement is the mean one. 

Available VOTABLE fields:

bibcodelist(y1-y2)
biblio
cel
cl.g
coo(opt)
coo_bibcode
coo_err_angle
coo_err_maja
coo_err_mina
coo_qual
coo_wavelength
coordinates
dec(opt)
dec_prec
diameter
dim
dim_angle
dim_bibcode
dim_incl
dim_majaxis
dim_minaxis
dim_qual
dim_wavelength
dimensions
distance
distance_result
einstein
fe_h
flux(filtername)
flux_bibcode(filtername)
flux_error(filtername)
flux_name(filtername)
flux_qual(filtername)
flux_system(filtername)
flux_unit(filtername)
fluxdata(filtername)

In [22]:
Simbad.add_votable_fields('sptype')
Simbad.add_votable_fields('flux(V)') # add the Vega unit Magnitude 
Simbad.add_votable_fields('flux_unit(V)')
Simbad.add_votable_fields('flux_system(V)')
#Simbad.add_votable_fields('ubv')  # Johnson UBV system

In [23]:
Simbad.get_field_description("otype")

standard name of the object type


In [24]:
Simbad.get_field_description("ubv")

UBV data in Johnson's UBV system 
              compiled by J.-Cl. Mermilliod from Institut d Astronomie de Lausanne (1973A&AS...71..413M)


In [25]:
Simbad.get_field_description("uvby")

The Strömgren uvby photometric system


In [26]:
all_tables=[]
all_objnames=[]
all_ra=[]
all_dec=[]
all_sp_type=[]
all_sp_qual=[]
all_mag_V=[]

for file in all_selected_calspec:
    fullfilename=os.path.join(path_sed_calspec,file)
    
    hdu = fits.open(fullfilename)
    img=hdu[0].data
    hd=hdu[0].header
    
    OBJNAME=hd["TARGETID"]
    print("==============",OBJNAME,"===========================")
    
    result_table = Simbad.query_object(OBJNAME)
    result_table.pprint(show_unit=True, max_width=80)
    
    ra = coord.Angle(result_table["RA"], unit=u.hour) # create an Angle object
    dec = coord.Angle(result_table["DEC"], unit=u.deg) # create an Angle object
    skycoord=SkyCoord(ra=ra,dec=dec)[0]
    
    thetype=result_table["SP_TYPE"][0]
    thequal=result_table["SP_QUAL"][0]
    #mag_V_vega=result_table["FLUX_V_2"][0]
    mag_V_vega=result_table["FLUX_V"][0]
        
    all_objnames.append(OBJNAME)
    all_tables.append(result_table)
    all_ra.append(skycoord.ra.degree)
    all_dec.append(skycoord.dec.degree)
    all_sp_type.append(thetype)
    all_sp_qual.append(thequal)
    all_mag_V.append(mag_V_vega)
    

============== HD009051 ===========================
 MAIN_ID        RA           DEC      ... FLUX_V FLUX_UNIT_V FLUX_SYSTEM_V
             "h:m:s"       "d:m:s"    ...  mag                            
--------- ------------- ------------- ... ------ ----------- -------------
HD   9051 01 28 46.5026 -24 20 25.443 ...   8.92           V          Vega
============== HD031128 ===========================
 MAIN_ID        RA           DEC      ... FLUX_V FLUX_UNIT_V FLUX_SYSTEM_V
             "h:m:s"       "d:m:s"    ...  mag                            
--------- ------------- ------------- ... ------ ----------- -------------
HD  31128 04 52 09.9097 -27 03 50.940 ...   9.14           V          Vega
============== HD074000 ===========================
 MAIN_ID        RA           DEC      ... FLUX_V FLUX_UNIT_V FLUX_SYSTEM_V
             "h:m:s"       "d:m:s"    ...  mag                            
--------- ------------- ------------- ... ------ ----------- -------------
HD  74000 08 40 50.

In [27]:
result_table

MAIN_ID,RA,DEC,RA_PREC,DEC_PREC,COO_ERR_MAJA,COO_ERR_MINA,COO_ERR_ANGLE,COO_QUAL,COO_WAVELENGTH,COO_BIBCODE,SP_TYPE,SP_QUAL,SP_BIBCODE,FLUX_V,FLUX_UNIT_V,FLUX_SYSTEM_V
,"""h:m:s""","""d:m:s""",,,mas,mas,deg,,,,,,,mag,,
object,str13,str13,int16,int16,float32,float32,int16,str1,str1,object,object,str1,object,float32,str1,object
HD 60753,07 33 27.3190,-50 35 03.311,14,14,0.032,0.038,90,A,O,2018yCat.1345....0G,B3IV,C,1969MNRAS.144...31B,6.68,V,Vega


In [28]:
t = Table()

In [29]:
t["OBJNAME"]=all_objnames
t["RA"]=all_ra
t["DEC"]=all_dec
t["SP_TYPE"]=all_sp_type
t["SP_QUAL"]=all_sp_qual
t["magV"]=all_mag_V
t["FILES"]=all_selected_calspec


In [30]:
t.show_in_notebook()  

idx,OBJNAME,RA,DEC,SP_TYPE,SP_QUAL,magV,FILES
0,HD009051,22.193760833333332,-24.340400833333334,G6/8IIIwF5,D,8.920000076293945,hd009051_stis_003.fits
1,HD031128,73.04129041666667,-27.06415,F3/5Vw,D,9.140000343322754,hd031128_stis_003.fits
2,HD074000,130.2116825,-16.345142499999998,F2,D,9.65999984741211,hd074000_stis_003.fits
3,HD106252,183.37295833333332,10.041635555555555,G0,E,--,hd106252_stis_004.fits
4,HD111980,193.31271999999998,-18.522224722222223,F7V,C,8.380000114440918,hd111980_stis_003.fits
5,HD116405,200.68801541666664,44.71497527777778,A0V,C,8.34000015258789,hd116405_stis_004.fits
6,HD14943,35.72781333333333,-51.092127500000004,A5V,C,5.8979997634887695,hd14943_stis_004.fits
7,HD158485,261.52015416666666,58.6518975,A4V,C,6.49399995803833,hd158485_stis_004.fits
8,HD159222,263.00413416666663,34.27114722222222,G1V,C,--,hd159222_stis_004.fits
9,HD160617,265.7055145833333,-40.320973611111114,G0,D,8.729999542236328,hd160617_stis_003.fits


In [31]:
t.write('table_summary_calspec_hd.fits', overwrite=True)  